# [Memory](https://langchain-ai.github.io/langgraph/concepts/memory/)
- LangGraph의 Memory는 AI가 대화 내용을 기억할 수 있게 해주는 기능입니다. 
- 사람이 대화할 때 이전에 말한 내용을 기억하는 것처럼, AI도 과거 대화를 기억해서 더 자연스러운 대화를 할 수 있게 해줍니다.


## 왜 Memory가 필요한가요?
> Memory가 없다면:
```text
    사용자: "내 이름은 김철수야"
    AI: "안녕하세요!"

    사용자: "내 이름이 뭐였지?"
    AI: "죄송해요, 모르겠습니다"
```
> Memory가 있다면:
```text
    사용자: "내 이름은 김철수야"
    AI: "안녕하세요 김철수님!"

    사용자: "내 이름이 뭐였지?"
    AI: "김철수님이라고 하셨어요!"
```


# ConversationBufferWindowMemory 예시
**최근 N개 메시지만 기억하는 효율적인 챗봇**

긴 대화에서는 모든 내용을 기억하면 메모리가 부족할 수 있어요. 그래서 최근 몇 개의 메시지만 기억하는 방법을 사용합니다!


In [ ]:
# 필요한 라이브러리 import
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

### 1. AI 모델 설정

In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### 2. 윈도우 메모리 노드 정의

In [16]:
from langgraph.graph import MessagesState

def chat_node_with_window(state: MessagesState, window_size=4):
    """최근 window_size 개의 메시지만 유지하는 함수"""
    messages = state["messages"]
    
    # 메시지가 너무 많으면 최근 것만 선택
    if len(messages) > window_size:
        # 시스템 메시지는 항상 유지하고, 나머지는 최근 것만
        system_messages = [msg for msg in messages if msg.type == "system"]
        other_messages = [msg for msg in messages if msg.type != "system"]
        
        # 최근 메시지만 선택
        recent_messages = other_messages[-window_size:]
        windowed_messages = system_messages + recent_messages
    else:
        windowed_messages = messages
    
    response = llm.invoke(windowed_messages)
    return {"messages": [response]}

### 3. 그래프 생성

In [17]:
from langgraph.graph import StateGraph, MessagesState, START, END

graph_window = StateGraph(MessagesState)
graph_window.add_node("chat", chat_node_with_window)
graph_window.add_edge(START, "chat")
graph_window.add_edge("chat", END)

### 4. 컴파일

In [18]:
from langgraph.checkpoint.memory import MemorySaver 

memory_window = MemorySaver()
app_with_window = graph_window.compile(checkpointer=memory_window)

print("Window Memory 챗봇이 준비되었습니다! (최근 4개 메시지만 기억)")

Window Memory 챗봇이 준비되었습니다! (최근 4개 메시지만 기억)


### 5. 테스트 

In [19]:
from langchain_core.messages import HumanMessage
import uuid

# 중요: 같은 thread_id를 사용해야 대화가 연결됩니다!
memory_id = str(uuid.uuid4())
# Window Memory 테스트: 오래된 정보를 기억할까요?
config_window = {"configurable": {"thread_id": memory_id}}

print("=== Window Memory 챗봇 테스트 ===")
print("많은 정보를 말해보고, 나중에 오래된 정보를 기억하는지 확인해보겠습니다!\n")

=== Window Memory 챗봇 테스트 ===
많은 정보를 말해보고, 나중에 오래된 정보를 기억하는지 확인해보겠습니다!



In [20]:
# 여러 정보 입력
messages_to_test = [
    "내 이름은 김철수야",
    "나는 30살이야", 
    "내 이름이 뭐였지?", # 기억함 
    "서울에 살고 있어",
    "취미는 축구야",
    "좋아하는 색깔은 파란색이야",
    "어제 영화를 봤어",
    "오늘은 운동을 했어",
    "내 이름이 뭐였지?",  # 오래전 정보 (기억 못할 수도 있음)
    "어제 뭐했다고 했지?"  # 최근 정보 (기억해야 함)
]

for i, msg in enumerate(messages_to_test, 1):
    print(f"{i}번째 대화: '{msg}'")
    result = app_with_window.invoke(
        {"messages": [HumanMessage(content=msg)]}, 
        config=config_window
    )
    print(f"AI: {result['messages'][-1].content}")
    print("-" * 40)

print("\n결과: 최근 4개 메시지만 기억해서 메모리가 효율적!")
print("오래된 정보(이름)는 잊어버릴 수 있지만, 최근 정보(어제 한 일)는 기억해요!")

1번째 대화: '내 이름은 김철수야'
AI: 안녕하세요, 김철수님! 어떻게 도와드릴까요?
----------------------------------------
2번째 대화: '나는 30살이야'
AI: 30살이시군요! 이 나이에 특별히 하고 싶은 목표나 계획이 있으신가요?
----------------------------------------
3번째 대화: '내 이름이 뭐였지?'
AI: 당신의 이름은 김철수입니다. 다른 질문이나 도움이 필요하신 부분이 있으면 말씀해 주세요!
----------------------------------------
4번째 대화: '서울에 살고 있어'
AI: 서울에 살고 계시군요! 서울은 다양한 문화와 맛있는 음식, 멋진 경치가 있는 도시죠. 서울에서 좋아하는 장소나 활동이 있으신가요?
----------------------------------------
5번째 대화: '취미는 축구야'
AI: 축구를 좋아하시군요! 축구는 정말 재미있고 팀워크를 중요시하는 스포츠죠. 어떤 팀을 응원하시나요? 또는 축구를 하시는 것도 좋아하시나요?
----------------------------------------
6번째 대화: '좋아하는 색깔은 파란색이야'
AI: 파란색은 정말 멋진 색깔이죠! 시원하고 안정감을 주는 느낌이 있어요. 파란색과 관련된 특별한 이유가 있나요? 아니면 파란색을 좋아하는 다른 이유가 있으신가요?
----------------------------------------
7번째 대화: '어제 영화를 봤어'
AI: 어떤 영화를 보셨나요? 영화의 장르나 내용이 궁금해요! 재미있었나요?
----------------------------------------
8번째 대화: '오늘은 운동을 했어'
AI: 운동을 하셨군요! 어떤 운동을 하셨나요? 운동 후 기분이 어떠셨나요?
----------------------------------------
9번째 대화: '내 이름이 뭐였지?'
AI: 죄송하지만, 이전 대화에서 이름을